In [2]:
#%pip install azure-cognitiveservices-vision-computervision
#%pip install pillow
#%pip install levenshtein
#%pip install jaro-winkler
#%pip install opencv-python
#%pip install numpy
#%pip install matplotlib

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import cv2
import numpy as np
import Levenshtein
from scipy.spatial import distance
import jaro

In [4]:
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [5]:
img = cv2.imread("test2.jpeg")

img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

img = cv2.GaussianBlur(img, (3,3), 100)

img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,819,33)

img = 255 - img 

#kernel = np.ones((3,3),np.uint8)
#img = cv2.dilate(img, kernel, iterations=0)

#img = cv2.resize(img,(1000,1000))
#cv2.imshow("img",img)
#cv2.waitKey()

cv2.imwrite("imagine_procesata.jpg", img)

True

In [6]:
# img = open("test1.png", "rb")
img = open("imagine_procesata.jpg", "rb")

read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
#print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
result = []
cutii = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)
            cutii.append(line.bounding_box)

succes in resolvarea
TEMELOR la
LABORA toarele de
Inteligentà Artificialà!


In [7]:
cuvinte_rezultate = []
for line in result:
    cuvinte_rezultate+=(line.split())

litere_rezultate = "".join(cuvinte_rezultate)

cuvinte_truth = ["Succes", "in", "rezolvarea", "tEMELOR", "la", "LABORAtoarele", "de", "Inteligenta", "Artificiala!"]
litere_truth = "SuccesinrezolvareatEMELORlaLABORAtoareledeInteligentaArtificiala!"

nr_elem_min = min(len(cuvinte_rezultate),len(cuvinte_truth))

print("(Levenshtein) Calitate nivel cuvant:",Levenshtein.ratio(cuvinte_rezultate,cuvinte_truth))
print("(Levenshtein) Calitate nivel caracter:",Levenshtein.ratio(litere_rezultate,litere_truth),"\n")

# La hamming trebuie length egal (?)
#print("(Hamming) Calitate nivel cuvant:",distance.hamming(cuvinte_rezultate,cuvinte_truth))
#print("(Hamming) Calitate nivel caracter:",distance.hamming(list(litere_rezultate),list(litere_truth)),"\n")

suma=0
for i in range(0,nr_elem_min):
    suma += jaro.jaro_winkler_metric(cuvinte_rezultate[i],cuvinte_truth[i])
print("(JaroWinkler) Calitate nivel cuvant:",suma/nr_elem_min)
print("(JaroWinkler) Calitate nivel caracter:",jaro.jaro_winkler_metric(litere_rezultate,litere_truth),"\n")

(Levenshtein) Calitate nivel cuvant: 0.3157894736842105
(Levenshtein) Calitate nivel caracter: 0.9230769230769231 

(JaroWinkler) Calitate nivel cuvant: 0.7288214254880921
(JaroWinkler) Calitate nivel caracter: 0.820940170940171 



In [8]:
'''cutii_truth = [
[60,293 ,1348,293, 1348,479, 60,479],
[118,565 ,1057,565, 1057,741, 118,741],
[74,897 ,1021,897, 1021,1045, 74,1045],
[95,1132 ,1469,1132, 1469,1379, 95,1379]
]'''
cutii_truth = [
[60.0,293.0 , 1348.0,479.0],
[118.0,565.0 , 1057.0,741],
[74.0,897.0 , 1021.0,1045.0],
[95.0,1132.0 , 1469.0,1379.0]
]
def iou(box1,box2): #Intersection over Union
    interX1=max(box1[0],box2[0])
    interY1=max(box1[1],box2[1])
    interX2=min(box1[2],box2[2])
    interY2=min(box1[3],box2[3])

    width_inner = abs(interX2-interX1)
    height_inner = abs(interY2-interY1)

    Arie_1si2 = width_inner * height_inner
    
    Arie_1 = (abs(box1[2]-box1[0]))*(abs(box1[3]-box1[1]))
    Arie_2 = (abs(box2[2]-box2[0]))*(abs(box2[3]-box2[1]))
    
    Arie_1sau2 = (Arie_1 + Arie_2) - Arie_1si2

    return Arie_1si2/Arie_1sau2

cutii_rezultate = []
for cutie in cutii:
    cutii_rezultate.append([cutie[0],cutie[1],cutie[4],cutie[5]])


for i,j in zip(cutii_truth,cutii_rezultate):
    print(iou(i,j))

0.3245341614906832
0.7444149966114822
0.8486115471331944
0.6291534263608731
